In [1]:
%cd ../../../

/Users/nseverin/MyData/Projects/Science/LLM/sasrec-bert4rec-recsys23


/Users/nseverin/MyData/Projects/Science/LLM/sasrec-bert4rec-recsys23/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from collections import defaultdict
import re
import numpy as np
import pandas as pd

def open_text(filename): 
    with open(filename) as f:
        return f.read()


def parse_file_llm(log_data):
    # Regex patterns
    config_pattern = re.compile(r"experiments(.+?\.yaml)")
    epoch_pattern = re.compile(r"Epoch: (\d+)/\d+ \| Train Total Loss: ([\d.]+)")
    metrics_pattern = re.compile(r"(Validation|Test) Metrics: ({.+?})")
    
    # Parse the log
    results = defaultdict(lambda: {"validation_metrics": [], "test_metrics": []})
    current_config = None
        
    for line in log_data.splitlines():
        config_match = config_pattern.search(line)
        if config_match:
            current_config = config_match.group(1)
        
        epoch_match = epoch_pattern.search(line)
        if epoch_match:
            print(epoch_match)
            epoch_num = int(epoch_match.group(1))
            loss = float(epoch_match.group(2))
        
        metrics_match = metrics_pattern.search(line)
        if metrics_match and current_config:
            metric_type = metrics_match.group(1).lower()  # "validation" or "test"
            metrics = eval(metrics_match.group(2))  # Convert string to dict
            results[current_config][f"{metric_type}_metrics"].append({"epoch": epoch_num, "loss": loss, **metrics})
    return results


def find_max(results, config, metric='NDCG@10'):
    def get_dct_epoch(lst, epoch):
        for x in lst:
            if x['epoch'] == epoch:
                return x
        return lst[-1]
        # raise Exception()
    
    def postprocess(lst):
        res = []
        for dct in lst:
            res.append({
                'NDCG@10': round(dct['NDCG@10'], 5),
                'Recall@10': round(dct['Recall@10'], 5),
                'epoch': dct['epoch']
            })
        return res
    
    best_val_dct = None
    for dct in results[config]['validation_metrics']:
        if best_val_dct is None:
            best_val_dct = dct
        elif dct[metric] > best_val_dct[metric]:
            best_val_dct = dct
    
    best_val_epoch = best_val_dct['epoch']
    
    return {
        'test': postprocess([get_dct_epoch(results[config]['test_metrics'], best_val_epoch)]),
        'validation': postprocess([best_val_dct]),
    }


PARAMS = ["config_file", "weighting_scheme", "alpha", "fine_tune_epoch", 'reconstruct_loss', "reconstruction_layer", 'weight_scale', 'use_down_scale','use_upscale','multi_profile','multi_profile_aggr_scheme','scale_guide_loss','user_profile_embeddings_files']


def create_dataframe(exp_data_with_max_val):
    # List to store extracted experiment data
    data = []
    
    # Parse experiment results
    for config_path, metrics in exp_data_with_max_val.items():
        # Extract config details from the filename
        config_file = config_path.split("/")[-1]  # Get only the filename
        config_name = config_file.replace(".yaml", "").replace('--1', '-4').lstrip('sasrec-').split("-")
        
        
        # {weighting_scheme}-{alpha}-{fine_tune_epoch}-{reconstruct_loss}-{reconstruction_layer}-{weight_scale}-{use_down_scale}-{use_upscale}-{multi_profile}-{multi_profile_aggr_scheme}-{scale_guide_loss}-{user_profile_embeddings_files}-{seed}.yaml
        weighting_scheme = (config_name[0])
        alpha = float(config_name[1])
        fine_tune_epoch = int(float(config_name[2]))
        reconstruct_loss = (config_name[3])
        reconstruction_layer = int(config_name[4])
        weight_scale = float(config_name[5])
        use_down_scale = (config_name[6])
        use_upscale = (config_name[7])
        multi_profile = (config_name[8])
        multi_profile_aggr_scheme = (config_name[9])
        scale_guide_loss = (config_name[10])
        user_profile_embeddings_files = (config_name[11])
        seed = int(config_name[12])
        
        # Extract validation and test metrics
        val_metrics = metrics.get("validation", [{}])[0]
        test_metrics = metrics.get("test", [{}])[0]
    
        # Append extracted values to data list
        data.append([
            config_file, weighting_scheme, alpha, fine_tune_epoch, reconstruct_loss, reconstruction_layer, weight_scale, use_down_scale,use_upscale,multi_profile,multi_profile_aggr_scheme,scale_guide_loss,user_profile_embeddings_files,seed,
            val_metrics.get("NDCG@10", None), val_metrics.get("Recall@10", None), val_metrics.get("epoch", None),
            test_metrics.get("NDCG@10", None), test_metrics.get("Recall@10", None), test_metrics.get("epoch", None)
        ])
    
    # Create DataFrame
    columns = PARAMS + ['seed',
        "val_NDCG@10", "val_Recall@10", "val_epoch", "test_NDCG@10", "test_Recall@10", "test_epoch"
        ]
    
    df = pd.DataFrame(data, columns=columns)
    return df


def apply_seed_info(df):
    df['params'] = df.apply(lambda row: tuple([row[param] for param in PARAMS[1:]]), axis=1)
    params2seeds = {}
    for params, seed in zip(df['params'], df['seed']):
        if params not in params2seeds:
            params2seeds[params] = []
        params2seeds[params].append(seed)
    df['all_seeds'] = df.apply(lambda row: (params2seeds[row['params']]), axis=1)
    return df

In [3]:
len('{weighting_scheme}-{alpha}-{fine_tune_epoch}-{reconstruct_loss}-{reconstruction_layer}-{weight_scale}-{use_down_scale}-{use_upscale}-{multi_profile}-{multi_profile_aggr_scheme}-{scale_guide_loss}-{user_profile_embeddings_files}-{seed}.yaml'.split('-'))

13

In [4]:
MODEL_NAME = 'sasrec'
DATASET = 'ml20m'
EXPERIMENT_NAME = 'baseline_cold_users'
SEED_FOLDER = 'other_seed'
SPLIT_NAME = 'cold_users'
MODE = 'LLM'


local_directory = f"experiments-2_0/results/{MODEL_NAME}/{DATASET}/{EXPERIMENT_NAME}" 

seed_folders = ['other_seed']

In [5]:
import os


results = {}
for seed_folder in seed_folders:
    cur_folder = os.path.join(local_directory, seed_folder)
    for file in os.listdir(cur_folder):
        log_data = open_text(f'{cur_folder}/{file}')
        cur_results = parse_file_llm(log_data)
        # if cur_results['simple']['validation_metrics'] == []:
        #     continue
        results.update(cur_results)
    
    
exp_data_with_max_val = {}
for config in results:
    exp_data_with_max_val[config] = find_max(results, config, metric='NDCG@10')
exp_data_with_max_val.keys()

<re.Match object; span=(0, 39), match='Epoch: 1/200 | Train Total Loss: 3.0161'>
<re.Match object; span=(0, 39), match='Epoch: 2/200 | Train Total Loss: 2.4973'>
<re.Match object; span=(0, 39), match='Epoch: 3/200 | Train Total Loss: 2.3072'>
<re.Match object; span=(0, 39), match='Epoch: 4/200 | Train Total Loss: 2.2381'>
<re.Match object; span=(0, 39), match='Epoch: 5/200 | Train Total Loss: 2.1983'>
<re.Match object; span=(0, 39), match='Epoch: 6/200 | Train Total Loss: 2.1713'>
<re.Match object; span=(0, 39), match='Epoch: 7/200 | Train Total Loss: 2.1510'>
<re.Match object; span=(0, 39), match='Epoch: 8/200 | Train Total Loss: 2.1346'>
<re.Match object; span=(0, 39), match='Epoch: 9/200 | Train Total Loss: 2.1212'>
<re.Match object; span=(0, 40), match='Epoch: 10/200 | Train Total Loss: 2.1099'>
<re.Match object; span=(0, 40), match='Epoch: 11/200 | Train Total Loss: 2.0998'>
<re.Match object; span=(0, 40), match='Epoch: 12/200 | Train Total Loss: 2.0910'>
<re.Match object; span=(0

dict_keys(['-2_0/configs/sasrec/ml20m/baseline_cold_users/other_seed/exponential-0.6-30-MSE-3-0.1-True-False-False-mean-True-gemma_short_large_umap_single-1.yaml', '-2_0/configs/sasrec/ml20m/baseline_cold_users/other_seed/exponential-0.6-30-MSE-3-0.1-True-False-False-mean-True-gemma_short_large_umap_single-256.yaml', '-2_0/configs/sasrec/ml20m/baseline_cold_users/other_seed/exponential-0.6-30-MSE-3-0.1-True-False-False-mean-True-gemma_short_large_umap_single-42.yaml', '-2_0/configs/sasrec/ml20m/baseline_cold_users/other_seed/exponential-0.5-30-MSE-3-0.1-True-False-False-mean-True-gemma_short_large_umap_single-1.yaml', '-2_0/configs/sasrec/ml20m/baseline_cold_users/other_seed/exponential-0.5-30-MSE-3-0.1-True-False-False-mean-True-gemma_short_large_umap_single-256.yaml', '-2_0/configs/sasrec/ml20m/baseline_cold_users/other_seed/exponential-0.5-30-MSE-3-0.1-True-False-False-mean-True-gemma_short_large_umap_single-42.yaml', '-2_0/configs/sasrec/ml20m/baseline_cold_users/other_seed/exponen

In [6]:
df = create_dataframe(exp_data_with_max_val)
df['weighting_scheme'] = df['weighting_scheme'].apply(lambda x: x if x != 'ttention' else 'attention')
df['weighting_scheme'] = df['weighting_scheme'].apply(lambda x: x if x != 'xponential' else 'exponential')
df

,config_file,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,multi_profile,multi_profile_aggr_scheme,scale_guide_loss,user_profile_embeddings_files,seed,val_NDCG@10,val_Recall@10,val_epoch,test_NDCG@10,test_Recall@10,test_epoch
0,exponential-0.6-30-MSE-3-0.1-True-False-False-...,exponential,0.6,30,MSE,3,0.1,True,False,False,mean,True,gemma_short_large_umap_single,1,0.18429,0.29069,180,0.04743,0.08233,180
1,exponential-0.6-30-MSE-3-0.1-True-False-False-...,exponential,0.6,30,MSE,3,0.1,True,False,False,mean,True,gemma_short_large_umap_single,256,0.18510,0.28971,168,0.05186,0.08822,168
2,exponential-0.6-30-MSE-3-0.1-True-False-False-...,exponential,0.6,30,MSE,3,0.1,True,False,False,mean,True,gemma_short_large_umap_single,42,0.18514,0.28824,199,0.05045,0.08678,199
3,exponential-0.5-30-MSE-3-0.1-True-False-False-...,exponential,0.5,30,MSE,3,0.1,True,False,False,mean,True,gemma_short_large_umap_single,1,0.05714,0.10777,196,0.02592,0.05371,196
4,exponential-0.5-30-MSE-3-0.1-True-False-False-...,exponential,0.5,30,MSE,3,0.1,True,False,False,mean,True,gemma_short_large_umap_single,256,0.18481,0.29172,184,0.04713,0.08222,184
5,exponential-0.5-30-MSE-3-0.1-True-False-False-...,exponential,0.5,30,MSE,3,0.1,True,False,False,mean,True,gemma_short_large_umap_single,42,0.18406,0.29141,118,0.05055,0.08708,118
6,exponential-0.4-30-MSE-3-0.1-True-False-False-...,exponential,0.4,30,MSE,3,0.1,True,False,False,mean,True,gemma_short_large_umap_single,1,0.18540,0.29297,181,0.05182,0.09107,181
7,exponential-0.4-30-MSE-3-0.1-True-False-False-...,exponential,0.4,30,MSE,3,0.1,True,False,False,mean,True,gemma_short_large_umap_single,256,0.11452,0.19518,186,0.04789,0.08818,186
8,exponential-0.4-30-MSE-3-0.1-True-False-False-...,exponential,0.4,30,MSE,3,0.1,True,False,False,mean,True,gemma_short_large_umap_single,42,0.18304,0.29034,150,0.04715,0.08138,150


In [7]:
df['weighting_scheme'].value_counts()

weighting_scheme
exponential    9
Name: count, dtype: int64

In [8]:
df = apply_seed_info(df)
df.sort_values(by='val_NDCG@10', ascending=False, inplace=True)
df

,config_file,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,multi_profile,...,user_profile_embeddings_files,seed,val_NDCG@10,val_Recall@10,val_epoch,test_NDCG@10,test_Recall@10,test_epoch,params,all_seeds
6,exponential-0.4-30-MSE-3-0.1-True-False-False-...,exponential,0.4,30,MSE,3,0.1,True,False,False,...,gemma_short_large_umap_single,1,0.18540,0.29297,181,0.05182,0.09107,181,"(exponential, 0.4, 30, MSE, 3, 0.1, True, Fals...","[1, 256, 42]"
2,exponential-0.6-30-MSE-3-0.1-True-False-False-...,exponential,0.6,30,MSE,3,0.1,True,False,False,...,gemma_short_large_umap_single,42,0.18514,0.28824,199,0.05045,0.08678,199,"(exponential, 0.6, 30, MSE, 3, 0.1, True, Fals...","[1, 256, 42]"
1,exponential-0.6-30-MSE-3-0.1-True-False-False-...,exponential,0.6,30,MSE,3,0.1,True,False,False,...,gemma_short_large_umap_single,256,0.18510,0.28971,168,0.05186,0.08822,168,"(exponential, 0.6, 30, MSE, 3, 0.1, True, Fals...","[1, 256, 42]"
4,exponential-0.5-30-MSE-3-0.1-True-False-False-...,exponential,0.5,30,MSE,3,0.1,True,False,False,...,gemma_short_large_umap_single,256,0.18481,0.29172,184,0.04713,0.08222,184,"(exponential, 0.5, 30, MSE, 3, 0.1, True, Fals...","[1, 256, 42]"
0,exponential-0.6-30-MSE-3-0.1-True-False-False-...,exponential,0.6,30,MSE,3,0.1,True,False,False,...,gemma_short_large_umap_single,1,0.18429,0.29069,180,0.04743,0.08233,180,"(exponential, 0.6, 30, MSE, 3, 0.1, True, Fals...","[1, 256, 42]"
5,exponential-0.5-30-MSE-3-0.1-True-False-False-...,exponential,0.5,30,MSE,3,0.1,True,False,False,...,gemma_short_large_umap_single,42,0.18406,0.29141,118,0.05055,0.08708,118,"(exponential, 0.5, 30, MSE, 3, 0.1, True, Fals...","[1, 256, 42]"
8,exponential-0.4-30-MSE-3-0.1-True-False-False-...,exponential,0.4,30,MSE,3,0.1,True,False,False,...,gemma_short_large_umap_single,42,0.18304,0.29034,150,0.04715,0.08138,150,"(exponential, 0.4, 30, MSE, 3, 0.1, True, Fals...","[1, 256, 42]"
7,exponential-0.4-30-MSE-3-0.1-True-False-False-...,exponential,0.4,30,MSE,3,0.1,True,False,False,...,gemma_short_large_umap_single,256,0.11452,0.19518,186,0.04789,0.08818,186,"(exponential, 0.4, 30, MSE, 3, 0.1, True, Fals...","[1, 256, 42]"
3,exponential-0.5-30-MSE-3-0.1-True-False-False-...,exponential,0.5,30,MSE,3,0.1,True,False,False,...,gemma_short_large_umap_single,1,0.05714,0.10777,196,0.02592,0.05371,196,"(exponential, 0.5, 30, MSE, 3, 0.1, True, Fals...","[1, 256, 42]"


In [9]:
df.to_csv('ml20m_llm_all_runs.csv')

In [10]:
def get_stats_seeds(df_all):
    def aggregate_mean(series):
        first_elem = list(series)[0]
        if isinstance(first_elem, str) or isinstance(first_elem, bool):
            return first_elem
        return series.mean()
        # print(df)
        # all_cols = set(df.columns)
        # df_mean = df.mean(numeric_only=True)
        # rest_cols = all_cols - set(df_mean.columns)
        # for col in rest_cols:
        #     df_mean[col] = list(df[col])[0]
        # return df_mean
    
    def aggregate_std(series):
        # print(series)
        first_elem = list(series)[0]
        if isinstance(first_elem, str) or isinstance(first_elem, bool):
            return first_elem
        if len(series) == 1:
            return 0
        return series.std()
        # all_cols = set(df.columns)
        # df_mean = df.std(numeric_only=True)
        # rest_cols = all_cols - set(df_mean.columns)
        # for col in rest_cols:
        #     df_mean[col] = list(df[col])[0]
        # return df_mean
    
    
    metric_cols = ["val_NDCG@10", "val_Recall@10", "val_epoch", "test_NDCG@10", "test_Recall@10"]
        
    grouped_df = df_all.drop(['config_file', 'all_seeds', 'seed'], axis=1).groupby('params')
    df_mean = grouped_df.agg(aggregate_mean)
    df_std = grouped_df.agg(aggregate_std)
    for col in metric_cols:
        df_mean[col + '__std'] = df_std[col]
    return df_mean


def reorder_cols(df):
    order = ['val_epoch', 'val_epoch__std'] + PARAMS[1:] + ['val_NDCG@10', 'val_NDCG@10__std', 'val_Recall@10', 'val_Recall@10__std', 'test_NDCG@10', 'test_NDCG@10__std', 'test_Recall@10', 'test_Recall@10__std']
    return df[order]


df_all = df[df['all_seeds'].apply(len) == 3]
df_final = get_stats_seeds(df_all)
df_final = reorder_cols(df_final)
df_final.sort_values(by='val_NDCG@10', ascending=False, inplace=True)
df_final

,val_epoch,val_epoch__std,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,...,scale_guide_loss,user_profile_embeddings_files,val_NDCG@10,val_NDCG@10__std,val_Recall@10,val_Recall@10__std,test_NDCG@10,test_NDCG@10__std,test_Recall@10,test_Recall@10__std
params,,,,,,,,,,,,,,,,,,,,,
"(exponential, 0.6, 30, MSE, 3, 0.1, True, False, False, mean, True, gemma_short_large_umap_single)",182.333333,15.631165,exponential,0.6,30.0,MSE,3.0,0.1,True,False,...,True,gemma_short_large_umap_single,0.184843,0.000480,0.289547,0.001233,0.049913,0.002263,0.085777,0.003071
"(exponential, 0.4, 30, MSE, 3, 0.1, True, False, False, mean, True, gemma_short_large_umap_single)",172.333333,19.502137,exponential,0.4,30.0,MSE,3.0,0.1,True,False,...,True,gemma_short_large_umap_single,0.160987,0.040259,0.259497,0.055715,0.048953,0.002510,0.086877,0.004975
"(exponential, 0.5, 30, MSE, 3, 0.1, True, False, False, mean, True, gemma_short_large_umap_single)",166.000000,42.000000,exponential,0.5,30.0,MSE,3.0,0.1,True,False,...,True,gemma_short_large_umap_single,0.142003,0.073495,0.230300,0.106114,0.041200,0.013343,0.074337,0.018028


In [23]:
df_final.to_csv('BEAUTY_ALPHA_EXPS.csv')

In [22]:
df[df['params'] == ('exponential', 0.8, 30, 'MSE', 1, 0.1, 'True', 'False', 'False', 'mean', 'False', 'gemma_short_large_umap_single')]

,config_file,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,multi_profile,...,user_profile_embeddings_files,seed,val_NDCG@10,val_Recall@10,val_epoch,test_NDCG@10,test_Recall@10,test_epoch,params,all_seeds


In [32]:
df['params'][0]

('mean',
 0.5,
 50,
 'MSE',
 1,
 0.1,
 'True',
 'False',
 'False',
 'mean',
 'False',
 'gemma_short_large_umap_single')

In [30]:
df[df['all_seeds'].apply(len) != 3]

,config_file,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,multi_profile,...,user_profile_embeddings_files,seed,val_NDCG@10,val_Recall@10,val_epoch,test_NDCG@10,test_Recall@10,test_epoch,params,all_seeds


In [10]:
df_final['weighting_scheme'].value_counts()

weighting_scheme
mean           16
attention      15
exponential     9
Name: count, dtype: int64

In [14]:
df_final[df_final['scale_guide_loss'] == 'False']

,val_epoch,val_epoch__std,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,...,scale_guide_loss,user_profile_embeddings_files,val_NDCG@10,val_NDCG@10__std,val_Recall@10,val_Recall@10__std,test_NDCG@10,test_NDCG@10__std,test_Recall@10,test_Recall@10__std
params,,,,,,,,,,,,,,,,,,,,,
"(mean, 0.5, 6, MSE, 1, 0.1, False, True, False, mean, False, gemma_short_large_umap_single)",14.000000,0.000000,mean,0.50,6.0,MSE,1.0,0.1,False,True,...,False,gemma_short_large_umap_single,0.099597,0.000808,0.183400,0.000946,0.059867,0.000621,0.115200,0.000375
"(mean, 0.5, 4, RMSE, 2, 0.1, False, True, False, mean, False, gemma_long_large_umap_single)",9.333333,1.527525,mean,0.50,4.0,RMSE,2.0,0.1,False,True,...,False,gemma_long_large_umap_single,0.099523,0.000072,0.182587,0.000343,0.059717,0.000998,0.114783,0.001294
"(mean, 0.5, 6, MSE, 1, 0.1, False, True, False, mean, False, gemma_long_large_umap_single)",14.000000,0.000000,mean,0.50,6.0,MSE,1.0,0.1,False,True,...,False,gemma_long_large_umap_single,0.099293,0.000384,0.183760,0.000537,0.059667,0.000748,0.114967,0.000718
"(exponential, 0.8, 8, RMSE, 2, 0.1, False, True, False, mean, False, gemma_long_large_umap_single)",12.666667,1.527525,exponential,0.80,8.0,RMSE,2.0,0.1,False,True,...,False,gemma_long_large_umap_single,0.099367,0.000376,0.183277,0.000603,0.059640,0.000927,0.114837,0.001643
"(exponential, 0.8, 8, RMSE, 1, 0.1, False, True, False, mean, False, gemma_short_large_umap_single)",13.000000,1.732051,exponential,0.80,8.0,RMSE,1.0,0.1,False,True,...,False,gemma_short_large_umap_single,0.099387,0.000116,0.183037,0.000872,0.059383,0.000947,0.114020,0.000931
"(exponential, 0.5, 8, RMSE, 2, 0.1, False, True, False, mean, False, gemma_short_large_umap_single)",13.666667,0.577350,exponential,0.50,8.0,RMSE,2.0,0.1,False,True,...,False,gemma_short_large_umap_single,0.099817,0.000266,0.183417,0.001399,0.059373,0.001048,0.114633,0.002053
"(exponential, 0.8, 8, MSE, 1, 0.1, False, True, False, mean, False, gemma_long_large_umap_single)",14.666667,1.154701,exponential,0.80,8.0,MSE,1.0,0.1,False,True,...,False,gemma_long_large_umap_single,0.099327,0.000539,0.183563,0.000333,0.059353,0.000945,0.114333,0.000127
"(mean, 0.65, 8, RMSE, 1, 0.1, False, True, False, mean, False, gemma_short_large_umap_single)",14.000000,0.000000,mean,0.65,8.0,RMSE,1.0,0.1,False,True,...,False,gemma_short_large_umap_single,0.099287,0.000652,0.183017,0.001596,0.059330,0.000872,0.113890,0.000656
"(mean, 0.8, 6, RMSE, 1, 0.1, False, True, False, mean, False, gemma_short_large_umap_single)",14.000000,0.000000,mean,0.80,6.0,RMSE,1.0,0.1,False,True,...,False,gemma_short_large_umap_single,0.099290,0.000521,0.183927,0.000652,0.059287,0.001598,0.114723,0.001650


In [40]:
df['params'].value_counts()

params
(mean, 0.7, 12, MSE, 1, 0.1, True, False, False, mean, True, gemma_short_large_umap_single)         3
(mean, 0.7, 12, MSE, 2, 0.1, True, False, False, mean, False, gemma_long_large_umap_single)         3
(mean, 0.6, 6, RMSE, 2, 0.1, True, False, False, mean, True, gemma_long_large_umap_single)          3
(attention, 0.5, 6, RMSE, 1, 0.1, True, False, False, mean, False, gemma_long_large_umap_single)    3
(mean, 0.7, 12, MSE, 2, 0.1, True, False, False, mean, False, gemma_long_large_single)              3
                                                                                                   ..
(mean, 0.6, 12, MSE, 1, 0.1, True, False, False, mean, False, gemma_long_large_umap_single)         1
(mean, 0.7, 6, RMSE, 1, 0.1, False, False, False, mean, True, gemma_long_large_umap_single)         1
(attention, 0.5, 6, MSE, 2, 0.1, True, False, False, mean, False, gemma_long_large_single)          1
(attention, 0.6, 6, MSE, 2, 0.1, False, False, False, mean, True, gemma_lon

In [18]:
df['']

,config_file,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,multi_profile,...,user_profile_embeddings_files,seed,val_NDCG@10,val_Recall@10,val_epoch,test_NDCG@10,test_Recall@10,test_epoch,params,all_seeds
663,mean-0.8-6-RMSE-1-0.1-False-True-False-mean-Fa...,mean,0.80,6,RMSE,1,0.1,False,True,False,...,gemma_short_large_umap_single,42,0.09987,0.18422,14,0.06111,0.11661,14,"(mean, 0.8, 6, RMSE, 1, 0.1, False, True, Fals...",[42]
837,exponential-0.8-8-MSE-2-0.1-False-True-False-m...,exponential,0.80,8,MSE,2,0.1,False,True,False,...,gemma_long_large_umap_single,42,0.09898,0.18188,14,0.06083,0.11617,14,"(exponential, 0.8, 8, MSE, 2, 0.1, False, True...",[42]
779,exponential-0.65-8-RMSE-2-0.1-False-True-False...,exponential,0.65,8,RMSE,2,0.1,False,True,False,...,gemma_short_large_umap_single,42,0.09998,0.18356,14,0.06072,0.11579,14,"(exponential, 0.65, 8, RMSE, 2, 0.1, False, Tr...",[42]
649,mean-0.8-6-MSE-2-0.1-False-True-False-mean-Tru...,mean,0.80,6,MSE,2,0.1,False,True,False,...,gemma_long_large_umap_single,42,0.09983,0.18389,14,0.06072,0.11615,14,"(mean, 0.8, 6, MSE, 2, 0.1, False, True, False...",[42]
467,exponential-0.5-8-RMSE-2-0.1-True-False-False-...,exponential,0.50,8,RMSE,2,0.1,True,False,False,...,gemma_short_large_umap_single,42,0.09985,0.18454,14,0.06072,0.11703,14,"(exponential, 0.5, 8, RMSE, 2, 0.1, True, Fals...",[42]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,exponential-0.8-8-RMSE-2-0.1-True-False-False-...,exponential,0.80,8,RMSE,2,0.1,True,False,False,...,gemma_long_large_single,42,0.09951,0.18465,19,0.05589,0.10964,19,"(exponential, 0.8, 8, RMSE, 2, 0.1, True, Fals...",[42]
792,exponential-0.8-4-MSE-1-0.1-False-True-False-m...,exponential,0.80,4,MSE,1,0.1,False,True,False,...,gemma_long_large_single,42,0.09851,0.18318,11,0.05586,0.10847,11,"(exponential, 0.8, 4, MSE, 1, 0.1, False, True...",[42]
25,attention-0.65-8-MSE-1-0.1-False-True-False-me...,attention,0.65,8,MSE,1,0.1,False,True,False,...,gemma_long_large_umap_single,42,0.09773,0.18112,18,0.05583,0.10835,18,"(attention, 0.65, 8, MSE, 1, 0.1, False, True,...",[42]
274,attention-0.8-6-MSE-2-0.1-True-False-False-mea...,attention,0.80,6,MSE,2,0.1,True,False,False,...,gemma_short_large_single,42,0.09796,0.18210,7,0.05579,0.10824,7,"(attention, 0.8, 6, MSE, 2, 0.1, True, False, ...",[42]


In [10]:
df[PARAMS[1:]].to_json('best_ml20m.json', index=False, orient="records")

In [42]:
df_final[PARAMS[1:]].to_json('best_beauty_3seeds.json', index=False, orient="records")

In [17]:
results.keys()

dict_keys(['-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-False-False-False-mean-False-gemma_long_large_umap_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-False-False-False-mean-False-gemma_short_large_umap_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-False-False-False-mean-True-gemma_long_large_umap_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-False-False-False-mean-True-gemma_short_large_umap_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-True-False-False-mean-False-gemma_long_large_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-True-False-False-mean-False-gemma_long_large_umap_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-True-False-Fal

In [23]:
results['-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-False-False-False-mean-False-gemma_long_large_umap_single-42.yaml']

{'validation_metrics': [{'epoch': 1,
   'loss': 10.828,
   'Recall@5': 0.004162330905306972,
   'NDCG@5': 0.0023390187078764875,
   'Recall@10': 0.006243496357960458,
   'NDCG@10': 0.00302255539974203,
   'Recall@20': 0.010851791288836034,
   'NDCG@20': 0.004198139363666408,
   'loss_recsys': np.float64(10.347532872800473),
   'loss_guide': np.float64(0.0)},
  {'epoch': 2,
   'loss': 5.2138,
   'Recall@5': 0.004013676230117437,
   'NDCG@5': 0.0024368647296024288,
   'Recall@10': 0.008324661810613945,
   'NDCG@10': 0.003826736836797046,
   'Recall@20': 0.014122194143005798,
   'NDCG@20': 0.0052561474396673165,
   'loss_recsys': np.float64(10.325713298938892),
   'loss_guide': np.float64(0.0)},
  {'epoch': 3,
   'loss': 4.8,
   'Recall@5': 0.006094841682770923,
   'NDCG@5': 0.003388410317555781,
   'Recall@10': 0.009216589861751152,
   'NDCG@10': 0.004382003442624782,
   'Recall@20': 0.015014122194143005,
   'NDCG@20': 0.005846106928100518,
   'loss_recsys': np.float64(10.315859370761448

In [22]:
df['config_file']

426    mean-0.7-12-MSE-1-0.1-True-False-False-mean-Tr...
11     attention-0.7-12-MSE-2-0.1-True-False-False-me...
88     mean-0.7-6-RMSE-1-0.1-True-False-False-mean-Tr...
283    mean-0.5-6-MSE-1-0.1-True-False-False-mean-Tru...
373    attention-0.5-12-MSE-1-0.1-True-False-False-me...
                             ...                        
73     mean-0.7-6-MSE-2-0.1-True-False-False-mean-Tru...
257    mean-0.7-12-MSE-2-0.1-True-False-False-mean-Tr...
234    mean-0.6-6-MSE-2-0.1-True-False-False-mean-Tru...
259    mean-0.7-12-MSE-2-0.1-True-False-False-mean-Tr...
2      attention-0.7-12-MSE-2-0.1-False-False-False-m...
Name: config_file, Length: 558, dtype: object

In [1]:
PARAMS

NameError: name 'PARAMS' is not defined

In [11]:
df_final

,val_epoch,val_epoch__std,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,...,scale_guide_loss,user_profile_embeddings_files,val_NDCG@10,val_NDCG@10__std,val_Recall@10,val_Recall@10__std,test_NDCG@10,test_NDCG@10__std,test_Recall@10,test_Recall@10__std
params,,,,,,,,,,,,,,,,,,,,,
"(mean, 0.65, 130, RMSE, 1, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",187.000000,19.924859,mean,0.65,130.0,RMSE,1.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.181537,0.000266,0.288197,0.001323,0.042590,0.000977,0.073347,0.002034
"(mean, 0.65, 30, RMSE, 1, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",196.333333,3.785939,mean,0.65,30.0,RMSE,1.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.181477,0.001445,0.287293,0.003120,0.042173,0.000328,0.073280,0.000868
"(mean, 0.8, 30, MSE, 1, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",178.000000,1.732051,mean,0.80,30.0,MSE,1.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.181333,0.001285,0.288867,0.002677,0.041200,0.001629,0.071700,0.003037
"(mean, 0.65, 130, MSE, 1, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",174.333333,25.106440,mean,0.65,130.0,MSE,1.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.181070,0.000131,0.287900,0.001914,0.042580,0.002975,0.074620,0.004700
"(mean, 0.8, 130, MSE, 1, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",168.000000,18.248288,mean,0.80,130.0,MSE,1.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.181033,0.000687,0.287797,0.000582,0.041260,0.001527,0.071750,0.003095
"(mean, 0.8, 100, RMSE, 1, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",178.000000,6.000000,mean,0.80,100.0,RMSE,1.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.180920,0.000560,0.286803,0.001107,0.040950,0.001562,0.070580,0.002951
"(exponential, 0.8, 30, MSE, 1, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",177.000000,18.520259,exponential,0.80,30.0,MSE,1.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.180740,0.000197,0.287647,0.001047,0.047890,0.000676,0.081850,0.001945
"(mean, 0.65, 100, MSE, 1, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",162.333333,15.567059,mean,0.65,100.0,MSE,1.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.180707,0.000315,0.286637,0.001048,0.042893,0.003585,0.074783,0.005056
"(exponential, 0.8, 130, RMSE, 1, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",180.666667,18.036999,exponential,0.80,130.0,RMSE,1.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.180600,0.000620,0.287590,0.000052,0.045393,0.002886,0.078397,0.003748


In [14]:
df_final[6:].to_csv('ml20m_llm_runs_FINAL_4_200_epochs.csv')

In [27]:
np.corrcoef(df_final['test_NDCG@10'].values[:15], df_final['val_NDCG@10'].values[:15])

array([[ 1.       , -0.5951883],
       [-0.5951883,  1.       ]])

In [13]:
df[df['params'] ==('exponential', 0.65, 50, 'MSE', 1, 0.1, 'True', 'False', 'False', 'mean', 'True', 'gemma_short_large_single')]

,config_file,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,multi_profile,...,user_profile_embeddings_files,seed,val_NDCG@10,val_Recall@10,val_epoch,test_NDCG@10,test_Recall@10,test_epoch,params,all_seeds
12,exponential-0.65-50-MSE-1-0.1-True-False-False...,exponential,0.65,50,MSE,1,0.1,True,False,False,...,gemma_short_large_single,1,0.17211,0.27639,98,0.04652,0.07940,98,"(exponential, 0.65, 50, MSE, 1, 0.1, True, Fal...","[1, 256, 42]"
13,exponential-0.65-50-MSE-1-0.1-True-False-False...,exponential,0.65,50,MSE,1,0.1,True,False,False,...,gemma_short_large_single,256,0.17094,0.27750,100,0.04522,0.07727,100,"(exponential, 0.65, 50, MSE, 1, 0.1, True, Fal...","[1, 256, 42]"
14,exponential-0.65-50-MSE-1-0.1-True-False-False...,exponential,0.65,50,MSE,1,0.1,True,False,False,...,gemma_short_large_single,42,0.17012,0.27362,99,0.04753,0.08052,99,"(exponential, 0.65, 50, MSE, 1, 0.1, True, Fal...","[1, 256, 42]"
